In [1]:
# get yolo v3 weights and coco dataset object names

!mkdir model
%cd model/
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names wget https://pjreddie.com/media/files/yolov3.weights
%cd ..

/content/model
--2021-01-20 16:30:54--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2021-01-20 16:30:55 (64.9 MB/s) - ‘yolov3.cfg’ saved [8342/8342]

--2021-01-20 16:30:55--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2021-01-20 16:30:55 (40.7 MB/s) - ‘coco.names’ saved [625/625]

--2021-01-20

In [ ]:
# import required libraries

import numpy as np
import argparse
import cv2
import os
import time
from google.colab import drive

In [2]:
# mount drive if neccessary
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# function to get boxes confidences and returning

def extract_boxes_confidences_classids(outputs, confidence, width, height):
    boxes = []
    confidences = []
    classIDs = []
    object_counts = np.zeros(85)

    for output in outputs:
        for detection in output:            
            scores = detection[5:]
            classID = np.argmax(scores)
            conf = scores[classID]
            
            if conf > confidence:
                box = detection[0:4] * np.array([width, height, width, height])
                centerX, centerY, w, h = box.astype('int')

                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))

                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(conf))
                classIDs.append(classID)

    return boxes, confidences, classIDs

In [17]:
# draw bounding boxes and object names and counts

def draw_bounding_boxes(image, boxes, confidences, classIDs, idxs, colors):
    object_counts = np.zeros(85)

    if len(idxs) > 0:
        for i in idxs.flatten():
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]

            color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            #print(labels[classIDs[i]])
            object_counts[classIDs[i]] += 1
            text = "{}: {:.4f}".format(labels[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            #print(object_counts)
        offset = 0

        font = cv2.FONT_HERSHEY_SIMPLEX
        for i in range(len(labels)):
            if object_counts[i] != 0:
                print(labels[i])
                print(object_counts[i])
                offset += 40
                text = labels[i] + '->' + str(int(object_counts[i]))
                cv2.putText(image, text, (20, offset), font, 1, (255, 0, 0), 2)
                

    return image, object_counts

In [ ]:
# function to use opencv dnn module to get predictions from yolo v3 weights

def make_prediction(net, layer_names, labels, image, confidence, threshold):
    height, width = image.shape[:2]
    
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(layer_names)

    boxes, confidences, classIDs = extract_boxes_confidences_classids(outputs, confidence, width, height)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence, threshold)

    return boxes, confidences, classIDs, idxs

In [ ]:
# not used for future purposes

'''def count_objects(classIDs, idxs, counts):
    if len(idxs) > 0:
        for i in idxs.flatten():
            counts[classIDs[i]] += 1
    
        for i in range(len(labels)):
            if counts[classIDs[i]] > 0:
                print(labels[classIDs[i]])
                print(counts[classIDs[i]])
'''

In [24]:
# Main code and parameters to run object counting with defined parameters ob images and videos

weights = '/content/model/yolov3.weights'
config = '/content/model/yolov3.cfg'
labels = '/content/model/coco.names'

confidence = 0.30
threshold = 0.30

use_gpu = False
save = True
image_path = ''
video_path = ''
show = False

labels = open(labels).read().strip().split('\n')

colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

net = cv2.dnn.readNetFromDarknet(config, weights)

if use_gpu:
    print('Using GPU')
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


if image_path != '':
    image = cv2.imread(image_path)
    print(image.shape)
    boxes, confidences, classIDs, idxs = make_prediction(net, layer_names, labels, image, confidence, threshold)

    image, counts = draw_bounding_boxes(image, boxes, confidences, classIDs, idxs, colors)

    if show:
        cv2.imshow('YOLO Object Detection', image)
        cv2.waitKey(0)
    
    if save:
        cv2.imwrite('./image_output.png', image)

else:
    if video_path != '':
        cap = cv2.VideoCapture(video_path)
        if (cap.isOpened()== False):
            print("Error opening video stream or file")
    else:
        cap = cv2.VideoCapture(0)

    if save:
        width = int(cap.get(3))
        height = int(cap.get(4))
        fps = cap.get(cv2.CAP_PROP_FPS)
        name = 'video_output.mp4'
        out = cv2.VideoWriter(f'./{name}', cv2.VideoWriter_fourcc('M','J','P','G'), fps, (width, height))
  
    while cap.isOpened():
        
            ret, image = cap.read()

            if not ret:
                print('Video file finished.')
                break

            boxes, confidences, classIDs, idxs = make_prediction(net, layer_names, labels, image, confidence, threshold)

            image, counts = draw_bounding_boxes(image, boxes, confidences, classIDs, idxs, colors)

            if show:
                cv2.imshow('YOLO Object Detection', image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            if save:
                out.write(image)

    cap.release()
    if save:
        out.release()
cv2.destroyAllWindows()

person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
cell phone
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
cell phone
1.0
person
3.0
car
1.0
cell phone
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
2.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
2.0
car
1.0
person
3.0
car
1.0
person
3.0
car
1.0
perso

In [ ]:
#!cp "/content/drive/Shareddrives/hasib/cv2.cpython-36m-x86_64-linux-gnu.so" .

In [ ]:
#!cp "/content/drive/Shareddrives/hasib/cv2_gpu" .